In [11]:
import sys
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import copy
from common_rl_lib import *
sys.path.append('p3')
import gridworld as gw


In [12]:
"""Prediction Algorithms - using dynamics model
LA and DP algorithms for solving small MDPs directly or iteratively
from generated MDPs - to get the true value function and compare it to the sampling methods below
"""
class LASolver: # solve using linear algebra v = (1 - discount*dynamics)^-1*rewards
    def __init__(self, mdp, policy=None):
        self.mdp = mdp
        self.policy = policy # if None, then this is find the optimal policy
        if self.mdp.deterministic:
            self.det_solve()
        else:
            self.sto_solve()

class ValueIterationSolver:
    def __init__(self, mdp, policy=None, discount=0.9, threshold=1e-3):
        self.mdp = mdp
        self.policy = policy
        self.givenPolicy = policy is not None
        self.discount = discount
        self.values = SafeDict()
        self.threshold = threshold # threshold for convergence
        self.solved = False
        self.iterations = 0 # iterations completed
    
    def _improveValues(self): # internal function
        self.iterations += 1
        newValues = SafeDict()
        max_delta = 0
        for s in self.mdp.state_space():
            if self.givenPolicy:
                newValues[s] = self.computeQValueFromValues(s, self.policy.decide(s, self.mdp.action_space(s)))
            else:
                options = [ self.computeQValueFromValues(s, a) for a in self.mdp.action_space(s)]
                if len(options): # not terminal state
                    newValues[s] = max(options)
            error = np.abs(self.values[s] - newValues[s])
            if error > max_delta:
                max_delta = error
        self.values = newValues
        return max_delta
    
    def computeQValueFromValues(self, state, action):
        return sum([ prob * (self.mdp.getReward(state=state, action=action, newstate=sp) + (self.discount * self.values[sp])) 
                for sp, prob in self.mdp.getTransitions(state=state, action=action)])
    
    def slowCompute(self, state, action):
        parts = [ prob * (self.mdp.getReward(state=state, action=action, newstate=sp) + (self.discount * self.values[sp])) 
                for sp, prob in self.mdp.getTransitions(action=action,state=state)]
        #print '--res s:', state, 'a:', action, 'calc:', parts
        return sum(parts)
    
    def iterate(self, iterations=10): # iterates regardless of whether mdp is already solved
        delta = 0
        for _ in range(iterations):
            delta = self._improveValues()
        if delta < self.threshold: self.solved = True
        return delta # from last iteration
    
    def solve(self, threshold=None): # only solves in passed in threshold is smaller than solved threshold
        if self.solved and threshold is not None and threshold >= self.threshold:
            return -1 # mdp is already solved
        if threshold is not None:
            self.threshold = threshold
            self.solved = False
        converged = False
        delta = 0
        while not converged:
            delta = self._improveValues()
            converged = delta < self.threshold
        self.solved = True
        return self.iterations # total number of iterations completed
            
    def getValue(self, state):
        return self.values[state]
    
    def getPolicy(self):
        if not self.givenPolicy:
            if not self.solved: self.solve()
            self.policy = Policy(decision=self.computeActionFromValues)
        return self.policy
    
    def computeActionFromValues(self, state):
        actions = self.mdp.getPossibleActions(state)
        if not len(actions):
            return None
        values = [self.computeQValueFromValues(state, a) for a in actions]
        return max(zip(actions,values),key=lambda x: x[1])[0]

In [13]:
"""Run Value Iteration on Gridworld MDPs"""
for name in gridWorlds.keys():
    print name, ValueIterationSolver(GridMDP(name)).solve() # prints number of iterations until convergence

bridge 5
discount 30
book 16
maze 23
cliff2 17
cliff 21


In [14]:
"""Model Free Prediction Algorithms - for policy evaluation
Standard Monte Carlo - only adjusts value function at the end of the episode
TD(0) - bootstraps each step to incrementally improve value function
TD(lambda) - bootstraps with eligibility function, to change between MC and TD
"""
class MC:
    def __init__(self, policy=Policy(), alpha=0.1, discount=0.9):
        self.alpha = alpha
        self.discount = discount
        self.values = SafeDict()
        self.policy = policy
        self.episode = [] # keeps track of visited states in this episode
        self.gain = 0
        self.state = None
        
    def take_action(self, actions):
        return self.policy.decide(self.state, actions)
    
    def eval_action(self, newstate, reward):
        self.episode.append(newstate)
        self.state = newstate
        self.gain += reward
        return self.gain
    
    def reset(self): # for MC this is where value function is improved
        total_discount = 1
        while len(self.episode):
            s = self.episode.pop()
            self.values[s] += self.alpha * (total_discount * self.gain - self.values[s])
            total_discount *= self.discount
        self.episode = []
        self.gain = 0
        self.state = None

class TD:
    def __init__(self, policy=Policy(), alpha=0.1, discount=0.9):
        self.alpha = alpha
        self.discount = discount
        self.values = SafeDict() # value function
        self.state = None
        self.policy = policy
    
    def take_action(self, actions): # reward is from previous action
        return self.policy.decide(self.state, actions)
    
    def eval_action(self, newstate, reward):
        if self.state is not None:
            self.values[self.state] += self.alpha * (reward + self.discount * self.values[newstate] - self.values[self.state])
            exp_reward = self.values[self.state]
        self.state = newstate
        return self.values[self.state] # return metric (eg. expected reward, loss, etc.)
    
    def reset(self):
        self.state = None

class TD_lambda:
    def __init__(self, policy=Policy(), alpha=0.1, discount=0.9, lmbda=0.9, threshold=1e-6):
        self.alpha = alpha
        self.discount = discount
        self.lmbda = lmbda
        self.threshold = threshold # remove insignificantly small elig. states to improve performance
        self.values = SafeDict() # value function
        self.policy = policy
        self.eligibility = SafeDict()
        self.state = None
    
    def take_action(self, actions): # reward is from previous action
        return self.policy.decide(self.state, actions)
    
    def eval_action(self, newstate, reward):
        if self.state is not None:
            for s in self.eligibility.keys():
                self.eligibility[s] *= self.discount * self.lmbda
                if self.eligibility[s] < self.threshold: del self.eligibility[s] # remove insignificant states
            self.eligibility[self.state] += 1
            for s in self.eligibility.keys():
                self.values[s] += self.alpha * self.eligibility[s] * (reward + self.discount * self.values[newstate] - self.values[self.state])
        self.state = newstate
        return self.values[self.state]
    
    def reset(self): # reset at the end of an episode
        self.eligibility = SafeDict()
        self.state = None

In [ ]:
""" Run policy evaluation (prediction) """
learner = TD_lambda() # random policy
watcher = MC()
found_states = []
episodes = 10
printstep = episodes / 100 if episodes > 100 else 1
steps = 20
for i in range(1,episodes+1): # number of episodes
    episode_len = 0
    mdp.reset()
    learner.reset()
    watcher.reset()
    if i % printstep == 0: print '\nEpsiode', i, '\n   ',
    state = None
    reward = None
    done = len(mdp.action_space()) == 0
    for _ in range(steps): # steps per episode
        print mdp.state,
        if done: break
        state, reward, done = mdp.step(learner.take_action(mdp.action_space()))
        learner.eval_action(state, reward)
        watcher.eval_action(state, reward)
        #state = state_maker(observation) # only neede if ai gym env is used
        episode_len += 1
    if i % printstep == 0: print '\n--- len:', episode_len
    found_states.append(len(learner.values))

#plt.plot(found_states) # shouldn't really decrease, since policy doesn't change
#plt.set_title('Number of States in Value Function')
#plt.show()

In [ ]:
print learner.values
print watcher.values
print mdp.dynamics
print mdp.rewards

In [15]:
"""Model-Free Control
On policy: SARSA - similar to TD, but now with epsilon greedy action choices\
           SARSA(lambda) - similar to TD(lambda)
Off policy: Q-learning
"""
class SARSA: # on policy - so policy is run while improving
    def __init__(self, policy=None, alpha=0.1, discount=0.9, epsilon=0.1):
        self.alpha = alpha
        self.discount = discount
        self.explore = lambda: epsilon > np.random.rand() # epsilon-greedy - constant epsilon
        self.Q = SafeDict() # Q function
        self.policy = policy
        if self.policy is None:
            self.policy = self.getPolicy() # epsilon greedy policy
        self.action = None
        self.state = None # current
    
    def take_action(self, actions):
        if self.action is None:
            self.action = self.policy.decide(self.state, actions)
        return self.action # already decided at last evaluation
    
    def eval_action(self, newstate, reward, newactions):
        if self.state is not None:
            newaction = self.policy.decide(newstate, newactions) if len(newactions) else None
            self.Q[self.state, self.action] += self.alpha * (reward + self.discount * self.Q[newstate, newaction] - self.Q[self.state, self.action])
            self.action = newaction
        self.state = newstate
        return self.Q[self.state, self.action]
    
    def __chooseAction(self, state): # epsilon-greedy policy
        if not self.explore(): # exploit
            options = [(a, self.Q[s,a]) for s,a in [key for key in self.Q.keys() if key[0]==state]]
            if len(options):
                return max(options, key=lambda x: x[1])[0] # argmax
        return None # policy will pick a random action
    
    def getPolicy(self, epsilon=None): # updates current policy if a prior policy was given
        if epsilon is not None:
            self.epsilon = epsilon
        self.policy = Policy(decision=self.__chooseAction)
        return self.policy
    
    def reset(self): # reset at the end of an episode
        self.state = None
        self.action = None

class SARSA_lambda: # on policy - so policy is run while improving
    def __init__(self, policy=None, alpha=0.1, discount=0.9, epsilon=0.1, lmbda=0.9, threshold=1e-6):
        self.alpha = alpha
        self.discount = discount
        self.lmbda = lmbda
        self.explore = lambda: epsilon > np.random.rand() # epsilon-greedy - constant epsilon
        self.threshold = threshold # remove insignificantly small elig. states to improve performance
        self.Q = SafeDict() # Q function
        self.E = SafeDict() # eligibility
        self.policy = policy
        if self.policy is None:
            self.policy = self.getPolicy() # epsilon greedy policy
        self.action = None
        self.state = None # current
    
    def take_action(self, actions):
        if self.action is None:
            self.action = self.policy.decide(self.state, actions)
        return self.action # already decided at last evaluation
    
    def eval_action(self, newstate, reward, newactions):
        if self.state is not None:
            newaction = self.policy.decide(newstate, newactions) if len(newactions) else None
            for sa in self.E.keys():
                self.E[sa] *= self.discount * self.lmbda
                if self.E[sa] < self.threshold: del self.E[sa] # remove insignificant states
            self.E[self.state, self.action] += 1
            for sa in self.E.keys():
                self.Q[sa] += self.alpha * self.E[sa] * (reward + self.discount * self.Q[newstate, newaction] - self.Q[self.state, self.action])
            self.action = newaction
        self.state = newstate
        return self.Q[self.state, self.action]
    
    def reset(self): # reset at the end of an episode
        self.E = SafeDict()
        self.state = None
        self.prevstate = None
        self.prevaction = None
    
    def __chooseAction(self, state): # epsilon-greedy policy
        if not self.explore(): # exploit
            options = [(a, self.Q[s,a]) for s,a in [key for key in self.Q.keys() if key[0]==state]]
            if len(options):
                return max(options, key=lambda x: x[1])[0] # argmax
        return None # policy will pick a random action
    
    def getPolicy(self, epsilon=None): # updates current policy if a prior policy was given
        if epsilon is not None:
            self.epsilon = epsilon
        self.policy = Policy(decision=self.__chooseAction)
        return self.policy
    
class Q_learning: # off policy - does not choose action, merely observes
    def __init__(self, policy=Policy(), alpha=0.1, discount=0.9):
        self.alpha = alpha
        self.policy = policy
        self.discount = discount
        self.Q = SafeDict() # Q function
        self.prevstate = None
        self.prevaction = None
    
    def step(self, state, action, reward): # reward is from previous action into state
        if self.prevstate is not None and self.prevaction is not None: # initial move
            bestAction = self.__chooseAction(state)
            if bestAction is None:
                bestAction = action # if you don't know any better trust the action of the supervisor
            self.Q[self.prevstate, self.prevaction] += self.alpha * (reward + self.discount
                                                                     * self.Q[state, bestAction]
                                                                     - self.Q[self.prevstate, self.prevaction])
        self.prevstate = state
        self.prevaction = action
        return self.Q[state, action]
    
    def reset(self):
        self.prevstate = None
        self.prevaction = None
    
    def __chooseAction(self, state): # epsilon-greedy policy
        options = [(a, self.Q[s,a]) for s,a in [key for key in self.Q.keys() if key[0]==state]]
        if len(options):
            return max(options, key=lambda x: x[1])[0] # argmax
        return None # policy will pick a random action
    
    def getPolicy(self): # build exploitative policy from Q function
        self.policy = Policy(decision=self.__chooseAction)
        return self.policy
        

In [ ]:
"""Reset learner and data collection"""
learner = SARSA_lambda() # on-policy
watcher = Q_learning() # off-policy
test_mdp = GridMDP('cliff')
print test_mdp.gridName
gains = []
completed_episodes = 0

In [ ]:
"""Run Model-Free Control"""
episodes = 100
printstep = episodes / 100 if episodes > 100 else 1
steps = 50
for i in range(episodes): # number of episodes
    test_mdp.reset()
    learner.reset()
    watcher.reset()
    
    if i % printstep == 0: print 'Epsiode', i+completed_episodes,
    gain = 0
    episode_len = 0
    state = test_mdp.state
    learner.state = state # initial state
    watcher.state = state # initial state
    reward = None
    while True:
    #for _ in range(steps): # steps per episode
        #print '\t', test_mdp.state, 
        action = learner.take_action(test_mdp.action_space())
        #print action,
        state, reward, done = test_mdp.step(action)
        #print reward
        learner.eval_action(state, reward, test_mdp.action_space())
        watcher.step(state, action, reward)
        #observation, reward, done, info = env.step(np.array([learner.take_action(state, reward, actions)]))
        #state = state_maker(observation)
        gain += reward
        episode_len += 1
        #print learner.Q
        if done: break
    if i % printstep == 0: print 'len:', episode_len, '- gain:', gain#,'\n'
    gains.append(gain)
completed_episodes += episodes

In [ ]:
"""Compare Policies that were learned"""
plt.plot(gains) # shouldn't really decrease, since policy doesn't change
#plt.set_title('Gain')
plt.show()

if 0:
    #print 'Learner (SARSA)'
    for s in test_mdp.state_space():
        for a in test_mdp.action_space(s):
            print 'Q(' + str(s) + ',' + str(a) + ') = l', learner.Q[s,a], 'w', watcher.Q[s,a]
    #print '\nWatcher (Q-learner)'
    #for s,a in watcher.Q.keys():
    #    print 'Q(' + str(s) + ',' + str(a) + ') = ' + str(watcher.Q[s,a])
else:
    l_policy = learner.getPolicy(epsilon=0)
    w_policy = watcher.getPolicy()
    for s in test_mdp.state_space():
        actions = test_mdp.action_space(s)
        if len(actions):
            l_a = l_policy.decide(s, actions)
            w_a = w_policy.decide(s, actions)
            print 'state',s,'options', actions, 'l:', l_a, 'w:', w_a
            if l_a != w_a:
                print '\tlearner:', [learner.Q[s,a] for a in actions]
                print '\twatcher', [watcher.Q[s,a] for a in actions]

In [ ]:
"""Function Approximation - based on TD but now in continuous state space using a NN to approximate Q function with discrete action space,
Action out framework - so output is expected value of each possible action"""
def net_maker(actions, activator=None):
    if activator:
        return nn.Sequential(
            nn.Linear(3, 20),
            activator(),
            nn.Linear(20, 50),
            activator(),
            nn.Linear(50, 20),
            activator(),
            nn.Linear(20, len(actions)),
            )
    return nn.Sequential(
            nn.Linear(3, 20),
            nn.Linear(20, 50),
            nn.Linear(50, 20),
            nn.Linear(20, len(actions)), # action out
            )

class TD_net:
    def __init__(self, actions, activator=None, alpha=0.001, discount=0.9, epsilon=0.1):
        self.discount = discount
        self.explore = lambda: epsilon > np.random.rand() # epsilon-greedy - constant epsilon
        self.actions = actions # total action space
        self.model = net_maker(actions, activator)
        self.optimizer = optim.SGD(self.model.parameters(), lr=alpha, momentum=0.9)
        self.criterion = nn.MSELoss()
        self.state = None
        self.iter_counter = 0
    
    def sample_actions(self, expectations, actions):
        expectations -= expectations.min() # guarantee expectations are positive, also makes it impossible to pick worst action
        expectations /= np.sum(expectations) # l1 normalization
        options = zip(actions, np.cumsum(expectations))
        sample = np.random.sample()
        try:
            action = [o[0] for o in options if sample < o[1]][-1]
        except:
            print 'expectations', np.sum(expectations), expectations, sample
            raise Exception('Sampling failed')
        return action
    
    def take_action(self, actions):
        if self.state is None: # initial move
            action = np.random.choice(actions)
            return action
        expectations = self.copy_model(self.state).data.numpy()
        if self.explore():
            action = self.sample_actions(expectations, actions)
        else: # exploit
            action = actions[expectations.argmax()] # greedy
        return action
    
    def eval_action(self, newstate, reward):
        if self.iter_counter % 200 == 0:
            self.copy_model = copy.deepcopy(self.model) # update frozen net
            print 'cloned'
        self.iter_counter += 1
        newstate = Variable(torch.Tensor(newstate))
        if self.state is None: # initial move
            self.state = newstate
            return -1
        target = Variable(reward + self.discount*self.copy_model(newstate).data) # get target from frozen net
        self.optimizer.zero_grad()
        loss = self.criterion(self.model(self.state), target)
        loss.backward()
        self.optimizer.step()
        self.state = newstate
        return loss.data[0]
    
    def reset_state(self): # reset at the end of an episode
        #self.E = SafeDict()
        self.state = None

In [ ]:
"""Run Function Approximation Control
Loss seems to be blowing up - not sure why, but I'm probably using pytorch incorrectly.
"""
actions = action_maker()
learner = TD_net(actions, epsilon=1)# nn.PReLU)
gains = []
losses = []
initial_states = []
episodes = 1000
printstep = episodes / 100
steps = 50
for i in range(episodes): # number of episodes
    env.reset()
    learner.reset_state()
    if i and i % printstep == 0: print 'Epsiode', i, 
    gain, total_loss = 0, 0
    state = None
    for _ in range(steps): # steps per episode
        action = learner.take_action(actions)
        state, reward, done, info = env.step(np.array([action]))
        state = state[np.newaxis,:]
        loss = learner.eval_action(state, reward)
        if loss < 0: 
            total_loss += 1 # compensate for loss flag
            initial_states.append(state[0])
            print state, # print initial state
        print loss
        gain += reward
        total_loss += loss
        if loss > 1000:
            print '***Terminating episode, loss:', loss
            break
        if np.isnan(loss): 
            raise Exception('Loss is nan')
    avg_loss = total_loss / steps
    if i and i % printstep == 0: print '- gain:', gain, '- avg loss:', avg_loss
    gains.append(gain)
    losses.append(avg_loss)

plt.plot(gains, 'g') # shouldn't really decrease, since policy doesn't change
plt.plot(losses[::steps], 'r')
#plt.set_title('Gain')
plt.show()

In [ ]:
inits = np.vstack(initial_states)
losses = np.array([0] + losses)[:,np.newaxis]
print inits.shape[0], 'episodes'

In [ ]:
stats = np.hstack([inits, losses])
print stats